In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import matplotlib.pyplot as plt

In [2]:
titanic_train = pd.read_csv('titanic_data/train.csv')
titanic_train.tail()
titanic_test_X = pd.read_csv("titanic_data/test.csv")
titanic_test_X.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [8]:
# sex = {'Sex':{'male': 1, 'female': 0}}
# titanic_train.replace(sex, inplace=True)
titanic_train_X = titanic_train.loc[:, titanic_train.columns != 'Survived']
titanic_train_y = titanic_train['Survived'].reset_index(drop=True)

In [12]:
# titanic_train_X = pd.get_dummies(titanic_train_X, columns=['Sex'], drop_first=True)
titanic_train_X.Cabin.dropna()

1              C85
3             C123
6              E46
10              G6
11            C103
          ...     
871            D35
872    B51 B53 B55
879            C50
887            B42
889           C148
Name: Cabin, Length: 204, dtype: object

In [13]:
titanic_train_X[titanic_train_X['Pclass'] == 1 & titanic_train_X['Cabin'].isna()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
30,31,1,"Uruchurtu, Don. Manuel E",male,40.0,0,0,PC 17601,27.7208,NaN,C
34,35,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C
35,36,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,NaN,S
64,65,1,"Stewart, Mr. Albert A",male,NaN,0,0,PC 17605,27.7208,NaN,C
83,84,1,"Carrau, Mr. Francisco M",male,28.0,0,0,113059,47.1000,NaN,S
155,156,1,"Williams, Mr. Charles Duane",male,51.0,0,1,PC 17597,61.3792,NaN,C
168,169,1,"Baumann, Mr. John D",male,NaN,0,0,PC 17318,25.9250,NaN,S
187,188,1,"Romaine, Mr. Charles Hallace (""Mr C Rolmane"")",male,45.0,0,0,111428,26.5500,NaN,S
256,257,1,"Thorne, Mrs. Gertrude Maybelle",female,NaN,0,0,PC 17585,79.2000,NaN,C
258,259,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C


In [10]:
scalable_columns = titanic_train_X.select_dtypes(include=np.number).columns
scaler = StandardScaler()
titanic_scaled = titanic_train_X.copy()
titanic_scaled[scalable_columns]= scaler.fit_transform(titanic_train_X[scalable_columns])
titanic_scaled.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,8.910000e+02,8.910000e+02,7.140000e+02,8.910000e+02,8.910000e+02,8.910000e+02
mean,6.379733e-17,-8.772133e-17,2.388379e-16,4.386066e-17,5.382900e-17,3.987333e-18
std,1.000562e+00,1.000562e+00,1.000701e+00,1.000562e+00,1.000562e+00,1.000562e+00
min,-1.730108e+00,-1.566107e+00,-2.016979e+00,-4.745452e-01,-4.736736e-01,-6.484217e-01
25%,-8.650540e-01,-3.693648e-01,-6.595416e-01,-4.745452e-01,-4.736736e-01,-4.891482e-01
50%,0.000000e+00,8.273772e-01,-1.170488e-01,-4.745452e-01,-4.736736e-01,-3.573909e-01
75%,8.650540e-01,8.273772e-01,5.718310e-01,4.327934e-01,-4.736736e-01,-2.424635e-02
max,1.730108e+00,8.273772e-01,3.465126e+00,6.784163e+00,6.974147e+00,9.667167e+00


In [11]:
enc = OneHotEncoder(handle_unknown='ignore')
titanic_train_scaled = enc.fit_transform(titanic_scaled)
titanic_test_scaled = enc.fit_transform(titanic_test_X)

array([[1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [13]:
from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score

In [14]:
rfc = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
rfc.fit(titanic_train_scaled, titanic_train_y)

RandomForestClassifier(n_estimators=10, random_state=0)

In [16]:
y_pred = rfc.predict(titanic_test_scaled)

ValueError: X has 1549 features, but RandomForestClassifier is expecting 2971 features as input.